In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# URL for Berlin, Munich and Hamburg Wikipedia pages
url_BER = "https://en.wikipedia.org/wiki/Berlin"
url_MUN = "https://en.wikipedia.org/wiki/Munich"
url_HAM = "https://en.wikipedia.org/wiki/Hamburg"
url_list=[url_BER,url_MUN,url_HAM]

In [4]:
# An empty dataframeto fill with extracted information
German_cities=pd.DataFrame(columns=['cityname','country','latitude','longitude','population','population_timestamp'])

In [11]:
# Empty lists to fill with the extracted information
cityname=[]
country=[]
latitude=[]
longitude=[]
population=[]

In [15]:
from datetime import datetime
!pip install lat-lon-parser
from lat_lon_parser import parse

for url_city in url_list:
  response_city = requests.get(url_city)
  soup_city=BeautifulSoup(response_city.content, 'html.parser')
  city_name=soup_city.select("div.fn.org")[0].get_text()
  country_name=soup_city.select("td.infobox-data a")[0].get_text()
  latitude_city=soup_city.select("span.latitude")[0].get_text()
  longitude_city=soup_city.select("span.longitude")[0].get_text()
  population_city=soup_city.find(string="Population").find_next("td").get_text()
  # formatting population 
  population_city_cl=int(population_city.replace(",", ""))
  print(f'City:{city_name}, Country:{country_name}, lalitude:{latitude_city}, longitude:{longitude_city}, population:{population_city_cl}')
  print()
  # formatting latitude and logitude (changing from degrees to floats)
  a=parse(latitude_city)
  b=parse(longitude_city)
  today = datetime.today().strftime("%d.%m.%Y")
  # Adding information to the lists
  cityname.append(city_name)
  country.append(country_name)
  latitude.append(a)
  longitude.append(b)
  population.append(population_city_cl)

City:Berlin, Country:Germany, lalitude:52°31′12″N, longitude:13°24′18″E, population:3878100

City:Munich , Country:Germany, lalitude:48°08′15″N, longitude:11°34′30″E, population:1510378

City:Hamburg, Country:Germany, lalitude:53°33′N, longitude:10°00′E, population:1964021



In [16]:
# Filling the data frame with the extracted info:
German_cities['cityname']=cityname
German_cities['country']=country
German_cities['latitude']=latitude
German_cities['longitude']=longitude
German_cities['population']=population
German_cities['population_timestamp']=today

In [21]:
# Creating cities and population data frames with the same structure as SQL tables:
cities=pd.DataFrame(columns=['city_name','country_code','latitude','longitude'])
population=pd.DataFrame(columns=['city_id','population','timestamp_population'])

In [29]:
# Filling cities data frame with extracted information
cities['city_id']=[1,2,3]
cities['city_name']=German_cities['cityname']
cities['country_code']=['DE','DE','DE']
cities['latitude']=German_cities['latitude']
cities['longitude']=German_cities['longitude']

In [39]:
# Filling populatiuon data frame with extracted information
population['city_id']=[1,2,3]
population['population']=German_cities['population']
population['timestamp_population']=German_cities['population_timestamp']
population['timestamp_population']=pd.to_datetime(population['timestamp_population'],dayfirst=True)

In [47]:
# Defining a connection string
schema = "YOUR_SQL_DATABASE_NAME"
host = "11.11.111.111"
user = "root"
password = "YOUR_PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [49]:
# Writing cities data frame to SQL cities table
cities.to_sql('cities',if_exists='append',con=connection_string,index=False)

3

In [51]:
# Writing population data frame to SQL population table
population.to_sql('population',if_exists='append',con=connection_string,index=False)

3